In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw
# import netgen.gui

In [ ]:
a = 1.5 # width of hole in coil
l = 2.94 # length of aluminium plate
r = 0.25 # width of coil

# hights of coil
hc0 = 0.49
hc1 = 1.49

ha = 0.19 # hight of aluminium plate

# x-y coordinates of vertices for coil 
'''(Eckpunkte des die Spule umschließenden Quaders, wie sag ich das auf Englisch?)'''
coil_outer_vertices = [(l-a-2*r,0), (l,0), (l,a+2*r), (l-a-2*r,a+2*r)]

# x-y coordinates of centers of circles for corners
coil_circle_centers = [(l-a, 2*r), (l-2*r, 2*r), (l-2*r, a), (l-a, a)]

# x-y coordinates of vertices for hole
hole_vertices = [(0.18, 0.18), (1.26, 1.26)]

d = 0.5 # extra space between geometry and enclosing box

In [ ]:
geo = CSGeometry()


cc = coil_circle_centers.copy()

''' warum brauch ich '-1e-3' damit es funktioniert? '''
bp = coil_outer_vertices.copy()
bp[0] = (bp[0][0], -1e-3)

maxh_corners = 0.1
# corners
corners = []

'''
########## was ich gerne haette ##########
for i in range(4):
    corners += [(Cylinder(Pnt(cc[i]+(0,)), Pnt(cc[i]+(1,)), 2*r) - Cylinder(Pnt(cc[i]+(0,)), Pnt(cc[i]+(1,)), r)) * \
       OrthoBrick(Pnt(bp[i]+(hc0,)),Pnt(cc[i]+(hc1,)))]
####################
'''

'''########## was ich stattdessen mache ##########'''
corners += [(Cylinder(Pnt(cc[0]+(0,)), Pnt(cc[0]+(1,)), 2*r) - Cylinder(Pnt(cc[0]+(0,)), Pnt(cc[0]+(1,)), r)) * \
             OrthoBrick(Pnt(bp[0]+(hc0,)),Pnt(cc[0]+(hc1,))).mat('corner0').maxh(maxh_corners)]
corners += [(Cylinder(Pnt(cc[1]+(0,)), Pnt(cc[1]+(1,)), 2*r) - Cylinder(Pnt(cc[1]+(0,)), Pnt(cc[1]+(1,)), r)) * \
             OrthoBrick(Pnt(bp[1]+(hc0,))-Vec(2*r, 0,0),Pnt(cc[1]+(hc1,))+Vec(2*r,0,0)).mat('corner1').maxh(maxh_corners)]
corners += [(Cylinder(Pnt(cc[2]+(0,)), Pnt(cc[2]+(1,)), 2*r) - Cylinder(Pnt(cc[2]+(0,)), Pnt(cc[2]+(1,)), r)) * \
             OrthoBrick(Pnt(cc[2]+(hc0,)),Pnt(bp[2]+(hc1,))).mat('corner2').maxh(maxh_corners)]
corners += [(Cylinder(Pnt(cc[3]+(0,)), Pnt(cc[3]+(1,)), 2*r) - Cylinder(Pnt(cc[3]+(0,)), Pnt(cc[3]+(1,)), r)) * \
             OrthoBrick(Pnt(bp[3]+(hc0,))-Vec(0,2*r,0), Pnt(cc[3]+(hc1,))+Vec(0,2*r,0)).mat('corner3').maxh(maxh_corners)]
'''####################'''


# bricks
b0 = [(l-a,0), (l-r,2*r), (l-a,a+r), (l-a-2*r,2*r)]
b1 = [(l-2*r,r),(l,a), (l-2*r,a+2*r), (l-a-r,a)]
bricks = []
for i in range(4):
    bricks += [OrthoBrick(Pnt(b0[i]+(hc0,)), Pnt(b1[i]+(hc1,))).mat('brick{}'.format(i))]

plate = (OrthoBrick(Pnt(0,0,0), Pnt(l,l,ha)) -
            OrthoBrick(Pnt(hole_vertices[0]+(0,)), Pnt(hole_vertices[1]+(ha,)))).mat('plate')


box = OrthoBrick(Pnt(0-d, 0-d, 0-d), Pnt(bp[2][0]+d, l+d, hc1+d)).bc("outer")
air = (box - sum(bricks) - sum(corners) - plate).mat('air')
    

geo.Add(air)
geo.Add(plate)
for i in range(4):
    geo.Add(corners[i])
    geo.Add(bricks[i])
# generate mesh
mesh = Mesh(geo.GenerateMesh(maxh=0.5))
mesh.Curve(5)


# generate normalized impressed current vectors
n = lambda center: CoefficientFunction(sqrt((x-center[0])**2+(y-center[1])**2))
sd = {
    'corner0': (-(y-cc[0][1])/n(cc[0]),(x-cc[0][0])/n(cc[0]),0) ,
    'corner1': (-(y-cc[1][1])/n(cc[1]),(x-cc[1][0])/n(cc[1]),0) , 
    'corner2': (-(y-cc[2][1])/n(cc[2]),(x-cc[2][0])/n(cc[2]),0) ,
    'corner3': (-(y-cc[3][1])/n(cc[3]),(x-cc[3][0])/n(cc[3]),0) ,
    'brick0': (1,0,0) ,
    'brick1': (0,1,0) ,
    'brick2': (-1,0,0) ,
    'brick3': (0,-1,0)
}
source_norm = CoefficientFunction([sd[mat] if mat in sd.keys() else (0,0,0) for mat in mesh.GetMaterials()])

# for mat in mesh.GetMaterials():
#     if mat in sd.keys():
#         print(mat)
#     else:
#         print('0 on ', mat) 

Draw (source_norm, mesh, draw_surf=False)

In [ ]:
%matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np

def T0z(x,y,z, sor=-1):
    
    # check if z is in area of coil
    if (z>hc1 or z<hc0): return(0)
    
    # check if x is in area 1 or 2
    if (x>= coil_outer_vertices[0][0] and x<coil_circle_centers[0][0]):
        
        # check if y is in area D or E
        if (y>=coil_outer_vertices[0][1] and y<coil_circle_centers[0][1]):
            
            dist = lambda p: sqrt((p[0]-coil_circle_centers[0][0])**2+(p[1]-coil_circle_centers[0][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
        
        # check if y is in area C
        elif (y >= coil_circle_centers[0][1] and y < coil_circle_centers[3][1]):
            
            if (x <= coil_outer_vertices[0][0]+r): 
                return(sor*(x-coil_outer_vertices[0][0])/r)
            else: return (sor)
        
        # check if y is in area A or B
        elif (y >= coil_circle_centers[3][1] and y <= coil_outer_vertices[3][1]):
            
            dist = lambda p: sqrt((p[0]-coil_circle_centers[3][0])**2+(p[1]-coil_circle_centers[3][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
        
     
    # check if x is in area 3
    elif (x>= coil_circle_centers[0][0] and x<coil_circle_centers[1][0]):
        
        # check if y is in area E
        if (y>=coil_outer_vertices[0][1] and y < coil_outer_vertices[0][1]+r):
            return(sor*y/r)
        
        # check if y is in area B to D
        elif (y >= coil_outer_vertices[0][1]+r and y < coil_outer_vertices[3][1]-r): 
            return (sor)
        
        # check if y is in area A
        elif (y >= coil_outer_vertices[3][1]-r and y < coil_outer_vertices[3][1]):
            return(-sor*(y-coil_outer_vertices[3][1])/r)

    # check if x is in area 4 or 5
    elif (x >= coil_circle_centers[1][0] and x <= coil_outer_vertices[1][0]):
        
        # check if y is in area D or E
        if (y>=coil_outer_vertices[0][1] and y<coil_circle_centers[0][1]):
            
            dist = lambda p: sqrt((p[0]-coil_circle_centers[1][0])**2+(p[1]-coil_circle_centers[1][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
        
        # check if y is in area C
        elif (y >= coil_circle_centers[0][1] and y < coil_circle_centers[3][1]):
            
            if ( x >= coil_outer_vertices[1][0]-r): 
                return(-sor*(x-coil_outer_vertices[1][0])/r)
            
            else: return (sor)
        
        # check if y is in area A or B
        elif (y >= coil_circle_centers[3][1] and y <= coil_outer_vertices[3][1]):
            
            dist = lambda p: sqrt((p[0]-coil_circle_centers[2][0])**2+(p[1]-coil_circle_centers[2][1])**2)
            if dist((x,y)) > 2*r: return (0)
            elif dist((x,y)) < r: return (sor)
            else: return (sor*(2*r-dist((x,y)))/r)
            
    return (0)

# plot for control
fig = plt.figure()
ax = plt.axes(projection='3d')

x = y = np.arange(0, l+0.1, 0.05)
X, Y = np.meshgrid(x, y)
zs = np.array([T0z(x,y,(hc0+hc1)/2) for x,y in zip(np.ravel(X), np.ravel(Y))])
Z = zs.reshape(X.shape)

ax.scatter(X, Y, Z)
plt.title('z coordianate of impressed current vector potential')

In [ ]:
N = 500
marg  = 0.1
x_lin = np.linspace(coil_outer_vertices[0][0]-marg, coil_outer_vertices[1][0]+marg, N)
y_lin = np.linspace(coil_outer_vertices[0][1]-marg, coil_outer_vertices[3][1]+marg, N)
z_lin = np.linspace(hc0-marg, hc1+marg, int(N*(hc1-hc0+2*marg)/(coil_outer_vertices[1][0]-coil_outer_vertices[0][0]+2*marg)))
values = np.array([T0z(x,y,z) for x in x_lin for y in y_lin for z in z_lin])
values = values.reshape(len(x_lin),len(y_lin),len(z_lin))
func = VoxelCoefficient((coil_outer_vertices[0][0]-marg,coil_outer_vertices[0][1]-marg,hc0-marg), 
        (coil_outer_vertices[1][0]+marg,coil_outer_vertices[3][1]+marg,hc1+marg), values, linear=True)

Draw(func, mesh)

In [ ]:
help(VoxelCoefficient)